In [8]:
import numpy as np
import numba

import rotations
import test_rotations

## Testing conversions between different representations

### Roundtrips between unit quaternions and rotation matrices

In [9]:
for i in range(1000):
    test_rotations.test_matrix_quaternion()

In [10]:
for i in range(1000):
    test_rotations.test_quaternion_matrix()

### Roundtrips between unit quaternions and Euler vectors

In [11]:
for i in range(1000):
    test_rotations.test_quaternion_euler()

In [12]:
for i in range(1000):
    test_rotations.test_euler_quaternion()
    
# Note: Only works when Euler vector has -2pi < theta < +2pi

known_euler [-1.85290899  1.21164221  3.3365971 ]
euler [ 1.05452537 -0.68956838 -1.8989202 ]


AssertionError: 

### Full roundtrips between rotation matrices and Euler vectors

In [13]:
for i in range(1000):
    test_rotations.test_matrix_euler()

In [14]:
for i in range(1000):
    test_rotations.test_euler_matrix()

known_euler [-1.51886005 -2.58895058  2.59076504]
euler [ 0.8879863  1.513604  -1.5146648]


AssertionError: 

### Other tests

In [15]:
test_rotations.test_quaternion_from_euler()

In [16]:
for i in range(1000):
    test_rotations.test_matrix_from_euler()

In [17]:
test_rotations.test_update_euler()

## Rotations greater than pi are broken!!!

In [18]:
def roundtrip(euler):
    
    euler = np.copy(euler)
    
    rev = int(np.linalg.norm(euler) / np.pi)
    print("rev", rev)
    
    if rev > 0:
        euler /= rev*np.pi
    
    quaternion = rotations.quaternion_from_euler(euler)

    matrix = rotations.matrix_from_quaternion(quaternion)
    
    quaternion2 = rotations.quaternion_from_matrix(matrix)
    
    assert np.allclose(quaternion, quaternion2)
    
    euler2 = rotations.euler_from_quaternion(quaternion2)
    
    if rev>0:
        euler2 *= rev*np.pi
    
    return euler2

In [19]:
def test_euler_matrix():
    while True:
        known_euler = np.random.normal(loc=0.0, scale=2*np.pi, size=3)
        if np.linalg.norm(known_euler) < 5*np.pi:
            break

    euler = roundtrip(known_euler)

    check = np.allclose(known_euler, euler)

    if not check:
        print("known_euler", known_euler)
        print("euler", euler)

    assert check

In [37]:
test_euler_matrix()

rev 4


## Interpolation of rotations

In [38]:
def random_euler_and_SO3():
    # create a random rotation in axis-angle representation
    while True:
        euler = np.random.normal(loc=0.0, scale=1.0, size=3)
        if np.linalg.norm(euler) < 1*np.pi:
            break

    # convert to a matrix
    matrix = rotations.matrix_from_quaternion(rotations.quaternion_from_euler(euler))
    
    assert np.allclose(np.dot(matrix, matrix.T), np.eye(3))
    
    return euler, matrix

In [39]:
def interpolate_euler(euler_left, euler_right):
    return (euler_left + euler_right) / 2

In [40]:
#@numba.jit(numba.float64[:, :](numba.float64[:, :], numba.float64[:, :]), nopython=True)
def interpolate_matrix(matrix_left, matrix_right):
    Qd = np.dot(matrix_right, matrix_left.T)
    ed = rotations.euler_from_quaternion(rotations.quaternion_from_matrix(Qd))

    ed2 = ed / 2    
    Qd2 = rotations.matrix_from_quaternion(rotations.quaternion_from_euler(ed2))
    
    Qi = np.dot(Qd2, matrix_left)
    return Qi

In [278]:
def experiment():
    el, Ql = random_euler_and_SO3()
    er, Qr = random_euler_and_SO3()
    
    ei_raw = interpolate_euler(el, er)
    Qi_raw = rotations.matrix_from_quaternion(rotations.quaternion_from_euler(ei_raw))
    
    Qi = interpolate_matrix(Ql, Qr)
    
    rel_error = np.linalg.norm(Qi_raw - Qi) / np.linalg.norm(Qi)
    
    if rel_error > 1:
        print('el', el)
        print('er', er)
        print('norm ei_raw', np.linalg.norm(ei_raw))
    
    print(rel_error * 100, 'percent')
    print(np.linalg.norm(el) / np.pi, np.linalg.norm(er) / np.pi)
    return el, er, Qi, Qi_raw

In [319]:
el, er, Qi, Qi_raw = experiment()

el [-1.17536511 -0.46418604  2.61488688]
er [ 1.5442882   1.89001636 -0.1636673 ]
norm ei_raw 1.42982287065
163.298915341 percent
0.924446947878 0.778641864976


In [261]:
Ql = rotations.matrix_from_euler(el)
Qr = rotations.matrix_from_euler(er)
Qi = interpolate_matrix(Ql, Qr)
np.dot(Qi, Ql.T)

array([[ 0.50697046,  0.76058235, -0.40558037],
       [-0.29093025,  0.59390643,  0.75008982],
       [ 0.81138187, -0.26227778,  0.52236943]])

In [262]:
np.dot(Qr, Qi.T)

array([[ 0.50697046,  0.76058235, -0.40558037],
       [-0.29093025,  0.59390643,  0.75008982],
       [ 0.81138187, -0.26227778,  0.52236943]])

In [55]:
def SO3_from_theta(theta):
    matrix = np.array([[+np.cos(theta), -np.sin(theta), 0],
                       [+np.sin(theta), +np.cos(theta), 0],
                       [0, 0, 1]])
    return matrix

In [56]:
Ql = SO3_from_theta(2)
Qr = SO3_from_theta(4)
Qi = interpolate_matrix(Ql, Qr)
Qi

array([[-0.9899925 , -0.14112001,  0.        ],
       [ 0.14112001, -0.9899925 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [57]:
SO3_from_theta(3)

array([[-0.9899925 , -0.14112001,  0.        ],
       [ 0.14112001, -0.9899925 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [60]:
el, Ql = random_euler_and_SO3()
er, Qr = random_euler_and_SO3()

In [61]:
%timeit interpolate_euler(el, er)

1.95 µs ± 18.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [62]:
%timeit interpolate_matrix(Ql, Qr)

12.5 µs ± 507 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
